In [1]:
import requests
import json

USERNAME = "bbb"
PASSWORD = "."


In [5]:
def getToken(username, password):
    res = requests.post(
        "http://localhost:8000/get-token/",
        data={"username": "bbb", "password": "."},
        verify=False,
    )
    return res.json()["token"]


def makeRoom(token, title, subtitle=None):
    res = requests.post(
        "http://localhost:8000/rooms/",
        headers={"Authorization": "Token " + TOKEN},
        data={"subtitle": subtitle, "title": title},
    )
    return res


def makeEvent(token, room, event_type, args):
    res = requests.post(
        "http://localhost:8000/events/",
        headers={"Authorization": "Token " + TOKEN, "content-type": "application/json"},
        data=json.dumps({"room": room, "event_type":event_type, "args":args}),
    )
    return res

def makeTune(token):
    with open("test.mp3", 'rb') as fobj:
        res = requests.post(
            "http://localhost:8000/tunes/",
            headers={"Authorization": "Token " + TOKEN},
            files={'file': fobj},
        )
        return res
    
def getEvents(token, room, event_type, skip, limit):
    url =  "http://localhost:8000/events/"
    
    res = requests.get(url,
                       params={"room":room, "event_type":event_type, "skip":skip, "limit":limit }, 
                       headers={"Authorization": "Token " + TOKEN})
    return res

def getTuneMeta(token, tune_id):
    url = "http://localhost:8000/tunes/{}/meta/".format(tune_id)
    res = requests.get(url, headers={"Authorization": "Token " + TOKEN})
    return res

def getTuneData(token, tune_id):
    url = "http://localhost:8000/tunes/{}/data/".format(tune_id)
    res = requests.get(url, headers={"Authorization": "Token " + TOKEN})
    return res

def getUser(token, skip, limit):
    url = "http://localhost:8000/users/?skip={}&limit={}".format(skip, limit)
    res = requests.get(url, headers={"Authorization": "Token " + TOKEN})
    return res


def makeUser(username, password):
    url = "http://localhost:8000/users/"
    res = requests.post(url,
        data={"username": username, "password": password},
    )
    return res

TOKEN = getToken(USERNAME, PASSWORD)



In [69]:
TOKEN = getToken("ceee", "deee")

In [77]:
res = makeUser("ceeaaeq", "deaeaeq")

SEX

In [3]:
res = makeRoom(TOKEN, "cwneaeeeewar", "boop")
res.json()

{'id': 8,
 'title': 'cwneaeeeewar',
 'subtitle': 'boop',
 'creator': 2,
 'members': [2],
 'system_user': None}

In [12]:
res = makeEvent(TOKEN, 8, "U", {"type": "J", "is_accepted":True})

In [11]:
res.json()

{'id': 21,
 'room': 8,
 'creation_time': '2020-03-15T16:49:20.142638Z',
 'event_type': 'U',
 'author': 2,
 'parent_event_id': None,
 'args': {'type': 'J', 'is_accepted': True},
 'isDeleted': False}

In [ ]:
res = makeEvent(TOKEN, 8, "U", {"type": "L"})

In [14]:
res.json()

{'id': 22,
 'room': 8,
 'creation_time': '2020-03-15T16:54:13.024478Z',
 'event_type': 'U',
 'author': 2,
 'parent_event_id': None,
 'args': {'type': 'L'},
 'isDeleted': False}

In [23]:
res = makeEvent(TOKEN, 8, "MQ", {"queue": [1,2]})

In [24]:
res

<Response [200]>

In [31]:
res = makeEvent(TOKEN, 8, "PL", {"song_id": 1, "is_playing":True, "timestamp": 10.4})

In [32]:
res.json()

{'id': 29,
 'room': 8,
 'creation_time': '2020-03-15T16:58:35.334802Z',
 'event_type': 'PL',
 'author': 2,
 'parent_event_id': None,
 'args': {'song_id': 1, 'is_playing': True, 'timestamp': 10.4},
 'isDeleted': False}

In [35]:
res = makeEvent(TOKEN, 8, "U", {"type": "I", "users": [3,4]})

In [36]:
res.json()

{'id': 31,
 'room': 8,
 'creation_time': '2020-03-15T17:01:36.939659Z',
 'event_type': 'U',
 'author': 2,
 'parent_event_id': None,
 'args': {'type': 'I', 'users': [3, 4]},
 'isDeleted': False}

In [39]:
res = makeEvent(TOKEN, 8, "U", {"type": "C", "user": 3, "role": "A"})

In [40]:
res.json()

{'id': 35,
 'room': 8,
 'creation_time': '2020-03-15T17:05:13.539927Z',
 'event_type': 'U',
 'author': 2,
 'parent_event_id': None,
 'args': {'type': 'C', 'user': 3, 'role': 'A'},
 'isDeleted': False}

In [41]:
res = makeEvent(TOKEN, 8, "U", {"type": "K", "user": 2})

In [43]:
res.json()

{'id': 36,
 'room': 8,
 'creation_time': '2020-03-15T17:08:28.543836Z',
 'event_type': 'U',
 'author': 2,
 'parent_event_id': None,
 'args': {'type': 'K', 'user': 2},
 'isDeleted': False}

In [44]:
res = makeEvent(TOKEN, 8, "M", {"content": "sex"})

In [45]:
res.json()

{'id': 37,
 'room': 8,
 'creation_time': '2020-03-15T17:09:50.043437Z',
 'event_type': 'M',
 'author': 2,
 'parent_event_id': None,
 'args': {'content': 'sex'},
 'isDeleted': False}

In [46]:
res = makeEvent(TOKEN, 8, "PO", {"action": "PL", "song_id": 1, "is_playing":True, "timestamp": 10.4})

In [47]:
res.json()

{'id': 38,
 'room': 8,
 'creation_time': '2020-03-15T17:11:05.287432Z',
 'event_type': 'PO',
 'author': 2,
 'parent_event_id': None,
 'args': {'action': 'PL', 'song_id': 1, 'is_playing': True, 'timestamp': 10.4},
 'isDeleted': False}

In [48]:
res = makeEvent(TOKEN, 8, "PO", {"action": "MQ", "queue": [1 ,2]})

In [49]:
res.json()

{'id': 39,
 'room': 8,
 'creation_time': '2020-03-15T17:12:06.149680Z',
 'event_type': 'PO',
 'author': 2,
 'parent_event_id': None,
 'args': {'action': 'MQ', 'queue': [1, 2]},
 'isDeleted': False}

In [51]:
res = makeEvent(TOKEN, 8, "PO", {"action": "U", "type": "K" , "user": 4})

In [52]:
res.json()

{'id': 40,
 'room': 8,
 'creation_time': '2020-03-15T17:13:22.864144Z',
 'event_type': 'PO',
 'author': 2,
 'parent_event_id': None,
 'args': {'action': 'U', 'type': 'K', 'user': 4},
 'isDeleted': False}

In [82]:
res = makeTune(TOKEN)

In [92]:
res = getTuneMeta(TOKEN, 2)

In [15]:
res = getUser(TOKEN, 0, 5)

In [107]:
res = getTuneData(TOKEN,2)

In [71]:
res.json()

{'id': 13,
 'room': 3,
 'creation_time': '2020-03-14T23:17:47.988366Z',
 'event_type': 'U',
 'author': 2,
 'parent_event_id': None,
 'args': {'type': 'I', 'users': [5]},
 'isDeleted': False}

In [35]:
audio = mutagen.File("test.mp3", easy=True)
audio.tags

{'album': ['TunePocket Unlimited Royalty Free Music Library'], 'composer': ['www.tunepocket.com'], 'title': ['Drums Urban Percussion Intro 2 (Music For Video)'], 'artist': ['TunePocket.com Music Library'], 'albumartist': ['www.tunepocket.com'], 'genre': ['Electronic']}

In [39]:
audio.mime[0]

'audio/mp3'

In [40]:
audio.info.length

23.412125

In [44]:
audio["title"]

['Drums Urban Percussion Intro 2 (Music For Video)']